# Configuração de ambiente

Importação das bibliotecas:
- Numpy: para manipulação de arrays
- Pandas: para manipulação de dataframes
- Matplotlib: para visualização de dados
- Seaborn: para visualização de dados
- Warnings: para poder alterar a visualização de warnings do sistema

E dentre outras para o aprendizado supervisionado.

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

sns.set()

%matplotlib inline

# Leitura e visualização do dataset

Leitura do arquivo "dataset_clean.csv" através da função "read_csv" do Pandas e armazenamento do resultado na variável "df". Uso do parâmetro "sep" para informar qual o separador de colunas do arquivo e do parâmetro "encoding" para informar qual o tipo de codificação do arquivo.

In [3]:
csv_url = "https://github.com/viniciusgugelmin/data-science-3/blob/main/projects/valorant_stats/data/dataset_clean.csv?raw=true"

df = pd.read_csv(csv_url, sep=';', low_memory=False)

Exibição das 5 primeiras linhas do DataFrame através da função "head" do Pandas para verificar se o arquivo foi carregado corretamente e ter uma ideia do que ele contém.

In [4]:
df.head()

,name,rating,damage_round,headshots,headshot_percent,aces,clutches,flawless,first_bloods,assists,kd_ratio,kills_round,score_round,win_percent,agent_1,gun1_name
0,ShimmyXD,Radiant,135.8,161.0,24.9,0,140,80,161.0,703.0,1.07,0.7,208.8,59.6,Fade,Vandal
1,XSET Cryo,Radiant,170.3,316.0,28.3,2,122,94,316.0,206.0,1.35,1.0,270.6,65.8,Chamber,Vandal
2,PuRelittleone,Radiant,147.5,216.0,24.0,3,117,59,216.0,267.0,1.05,0.8,227.8,65.6,Yoru,Vandal
3,Boba,Radiant,178.2,235.0,37.3,3,83,49,235.0,157.0,1.40,1.0,277.0,62.8,Jett,Vandal
4,i love mina,Radiant,149.8,137.0,24.4,2,71,38,137.0,213.0,1.11,0.8,230.9,62.8,Jett,Vandal


# Explicação sobre o trabalho a ser realizado

O dataset escolhido tem como base informações sobre como os melhores jogadores de Valorant do Episódio 4, Ato 3, se saíram dentro nos seus jogos. 

As colunas a serem trabalhadas contém informações sobre o nome dentro do jogo, elo/rank, dano por rodada, eliminações com tiro na cabeça, percentual de eliminações com tiro na cabeça, rodadas em que eliminou todos sozinho, rodadas em que ficou sozinho e ganhou, em que todos do time eliminaram alguém, rodadas em que foi o primeiro a eliminar, assistência em eliminações, percentual de eliminações e mortes, pontuação por rodada, percentual de vitória, melhor agente e melhor arma.

O intuito deste estudo é extrairmos algumas informações dessa base de dados, inserirmos alguns gráficos e realizarmos um aprendizado de máquina para conseguirmos prever o elo/ranque de um jogador só com base em seus resultados dentro do jogo.

# Perguntas iniciais

- Qual o jogador com maior percentual de vitórias e o com o menor?

In [5]:
df_win_order = df.copy()

df_win_order = df_win_order.sort_values('win_percent', ascending=True)
df_win_order # clean 0 values

,name,rating,damage_round,headshots,headshot_percent,aces,clutches,flawless,first_bloods,assists,kd_ratio,kills_round,score_round,win_percent,agent_1,gun1_name
44755,JoqcyaN,Immortal 1,68.0,0.0,20.7,0,1,0,0.0,8.0,0.35,0.3,106.8,0.0,Brimstone,Ghost
44938,NeLÃ˜,Immortal 1,192.8,3.0,11.7,1,1,0,3.0,3.0,1.81,1.2,334.9,0.0,Raze,Spectre
44933,Xem,Immortal 1,172.8,1.0,8.2,0,1,0,1.0,8.0,0.89,0.8,267.0,0.0,Sova,Guardian
44930,Lobo Loco,Immortal 1,120.6,1.0,23.1,0,1,0,1.0,2.0,0.47,0.5,163.9,0.0,Jett,Vandal
8849,Mad,Immortal 1,94.4,3.0,18.5,0,1,0,3.0,14.0,0.59,0.5,146.2,0.0,Omen,Vandal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83445,ONI Cid,Immortal 1,172.8,7.0,31.9,0,2,5,7.0,10.0,1.82,0.9,259.6,100.0,Reyna,Vandal
53132,Dyna,Immortal 1,140.5,7.0,31.5,0,2,6,7.0,5.0,1.27,0.7,214.4,100.0,Reyna,Vandal
27248,cStar,Immortal 1,172.6,6.0,19.0,0,6,1,6.0,23.0,1.28,0.8,258.4,100.0,Raze,Vandal
68847,7K johnpirate,Immortal 1,208.2,12.0,23.2,0,5,2,12.0,14.0,1.48,1.1,327.4,100.0,Neon,Vandal
